## Create a file(s) for marker locations containing distance to trap
### Originally, all three files were combined but the file size was too big or GitHub
### Only hospitals is create in this code for testing purposes

In [1]:
library(tidyverse) # data wrangling
library(data.table) # fread
library(dplyr)# data wrangling 

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.0.9000     ✔ purrr   0.2.5     
✔ tibble  2.0.1          ✔ dplyr   0.7.6     
✔ tidyr   0.8.1          ✔ stringr 1.4.0     
✔ readr   1.1.1          ✔ forcats 0.3.0     
Warning message:
“package ‘tibble’ was built under R version 3.4.4”Warning message:
“package ‘purrr’ was built under R version 3.4.4”Warning message:
“package ‘dplyr’ was built under R version 3.4.4”Warning message:
“package ‘stringr’ was built under R version 3.4.4”── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘data.table’

The following objects are masked from ‘package:dplyr’:

    between, first, last

The following object is masked from ‘package:purrr’:

    transpose



In [2]:
hospitals <- fread("https://raw.githubusercontent.com/rsangole/capstone_project/master/data/raw/hospitals/hospitals.csv")
schools <- fread("https://raw.githubusercontent.com/rsangole/capstone_project/master/data/raw/schools/School%20Point%202014.csv")
senior <- fread("https://raw.githubusercontent.com/rsangole/capstone_project/master/data/raw/senior_centers/senior.centers.csv")
wnv_df <- fread("https://raw.githubusercontent.com/rsangole/capstone_project/master/data/processed/wnv.trap.date.rev3b.csv")

In [3]:
head(hospitals,2)
head(schools,2)
head(senior,2)

#table(senior$CITY)

glimpse(senior)


CFNAME,CFTYPE,CFSUBTYPE,ADDRESS,GNISCODE,COMMENT,SOURCE,Jurisdicti,Community,MERGEID,⋯,POINT_Y,ZCTA5CE10,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,GEOID,LATITUDE,LONGITUDE,community
Advocate Christ Medical Center,Hospital,Hospital,9340 S Kostner Ave,426825,,CGI,NA,Oak Lawn,11216,⋯,1841934,60453,17,031,822102,1,170318221021,+41.7239031,-087.7273882,NA
Palos Primary Care Center,Hospital,Hospital,15300 West Ave,2036658,,CL,NA,Orland Park,11376,⋯,1801018,60462,17,031,824116,4,170318241164,+41.6076791,-087.8676051,NA


mergeid,cftype,gniscode,cfname,cfsubtype,comment,source,jurisdicti,community,address,POINT_X,POINT_Y
10000,School,419204,Strassburg Elementary School,Public School,,ACGIL,SCHOOL DIST #168,Sauk Village,2002 E 223rd St,-87.56634,41.48341
10002,School,2035677,Saint Mark's Lutheran School,Non Public School,,ACG,ST MARK EV LUTH CH,Sauk Village,22012 Torrence Ave,-87.55686,41.48871


PROGRAM,SITE.NAME,HOURS.OF.OPERATION,ADDRESS,CITY,STATE,ZIP,PHONE,LOCATION,ZCTA5CE10,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,GEOID,LATITUDE,LONGITUDE,community
Regional Senior Center,Northeast,Mon - Fri 8:30 a.m. to 4:30 p.m. Sat 9:00 a.m. to 4:00 p.m.,2019 W. Lawrence Avenue,Chicago,IL,60625,312-744-0784,"2019 W Lawrence Avenue Chicago, IL 60625 (41.96872254686461, -87.68019238203728)",60625,17,031,040600,1,170310406001,+41.9670999,-087.6806413,LINCOLN SQUARE
Regional Senior Center,Southwest,Mon - Fri 8:30 a.m. to 4:30 p.m. Sat 9:00 a.m. to 4:00 p.m.,6117 S. Kedzie Avenue,Chicago,IL,60629,312-747-0440,"6117 S Kedzie Avenue Chicago, IL 60629 (41.78212991978889, -87.70319764951621)",60629,17,031,660302,2,170316603022,+41.7835374,-087.7020495,CHICAGO LAWN


Observations: 21
Variables: 18
$ PROGRAM            <chr> "Regional Senior Center", "Regional Senior Center"…
$ SITE.NAME          <chr> "Northeast", "Southwest", "Northwest", "Central We…
$ HOURS.OF.OPERATION <chr> "Mon - Fri 8:30 a.m. to 4:30 p.m. Sat 9:00 a.m. to…
$ ADDRESS            <chr> "2019 W. Lawrence Avenue", "6117 S. Kedzie Avenue"…
$ CITY               <chr> "Chicago", "Chicago", "Chicago", "Chicago", "Chica…
$ STATE              <chr> "IL", "IL", "IL", "IL", "IL", "IL", "IL", "IL", "I…
$ ZIP                <int> 60625, 60629, 60618, 60612, 60649, 60602, 60619, 6…
$ PHONE              <chr> "312-744-0784", "312-747-0440", "312-744-6681", "3…
$ LOCATION           <chr> "2019 W Lawrence Avenue\nChicago, IL 60625\n(41.96…
$ ZCTA5CE10          <chr> "60625", "60629", "60618", "60612", "60649", "6060…
$ STATEFP            <chr> "17", "17", "17", "17", "17", "17", "17", "17", "1…
$ COUNTYFP           <chr> "031", "031", "031", "031", "031", "031", "031", "…
$ TRACTCE            

In [4]:
# --- get chicago hospitals
hospitals$LONGITUDE <- as.numeric(hospitals$LONGITUDE)
hospitals$LATITUDE <- as.numeric(hospitals$LATITUDE)

hosp_df <- hospitals %>%    
    select(CFNAME,LONGITUDE, LATITUDE, Community, CFTYPE) %>%
    filter(Community %in% "Chicago")

hosp_df <- hosp_df %>%
  setNames(tolower(gsub(" ","_",names(.))) )

# --- get chicago schools
sch_df <- schools %>%    
    select(cfname,POINT_X, POINT_Y, community, cftype) %>%
    rename(LONGITUDE = POINT_X , LATITUDE = POINT_Y) %>%
    filter(community %in% "Chicago") 

sch_df <- sch_df %>%
  setNames(tolower(gsub(" ","_",names(.))) )

# --- get senior centers
senior$LONGITUDE <- as.numeric(senior$LONGITUDE)
senior$LATITUDE <- as.numeric(senior$LATITUDE)

senior$cfname <- c(paste0(senior$SITE.NAME, " ", senior$PROGRAM," ",senior$community)) # create a name for SC
senior$CFTYPE <- "Senior Center"

snr_df <- senior %>%    
    select(cfname,LONGITUDE, LATITUDE, CITY, CFTYPE) %>%
    rename(community = CITY)
    #filter(CITY %in% "Chicago")

snr_df <- snr_df %>%
  setNames(tolower(gsub(" ","_",names(.))) )



In [5]:
head(hosp_df,2)
head(sch_df,2)
head(snr_df,2)
#glimpse(wnv_df)

cfname,longitude,latitude,community,cftype
Cermak Hospital,-87.69757,41.84098,Chicago,Hospital
Hartgrove Hospital,-87.76369,41.86881,Chicago,Hospital


cfname,longitude,latitude,community,cftype
Lawrence Hall Youth Services,-87.70003,41.96984,Chicago,School
House of Good Shepherd School,-87.65736,41.95191,Chicago,School


cfname,longitude,latitude,community,cftype
Northeast Regional Senior Center LINCOLN SQUARE,-87.68064,41.96710,Chicago,Senior Center
Southwest Regional Senior Center CHICAGO LAWN,-87.70205,41.78354,Chicago,Senior Center


read in the wnv master file and get unique traps info to creat a masterfile of traps

In [6]:
master_trap <- wnv_df %>%
    select(trap_trap_name, loc_lat, loc_lng)

master_trap_nodup <- master_trap[!duplicated(master_trap),]

dim(master_trap)
dim(master_trap_nodup)

[1] 13631     3

[1] 194   3

# create 3 marker files with haversine distances to trap
- 1 master file is too big for github
- the first file is all traps to hospitals

## hospital distance

In [7]:
dist.from.trap <- inner_join(
  cbind(master_trap_nodup[,c("trap_trap_name","loc_lat","loc_lng")],dummy=1)
  ,cbind(hosp_df[,c("cfname","longitude","latitude", "cftype")],dummy=1)
) %>% dplyr::select(-dummy) %>% ## rowwise() %>% 
  mutate(dist.from.trap.m = 
    geosphere::distHaversine(p1=cbind(loc_lng,loc_lat)
                             ,p2=cbind(as.numeric(longitude),as.numeric(latitude))
    )) %>%
  mutate(dist.from.trap.ft = dist.from.trap.m * 3.28084) %>%
  mutate(dist.from.trap.mi = dist.from.trap.ft / 5280) %>%
  dplyr::select(-c(dist.from.trap.ft)) %>%
  #dplyr::select(-c(POINT_Y,POINT_X,loc_lng,loc_lat,dist.from.trap.m)) 
  dplyr::select(-c(dist.from.trap.m)) 
  
  hosp2trap_dist <- dist.from.trap  %>%
  setNames(tolower(gsub(" ","_",names(.))) )

#round the miles to the nearest tenth
hosp2trap_dist$dist_round_mi <- round(hosp2trap_dist$dist.from.trap.mi, digits=1)

hosp2trap_dist <- hosp2trap_dist %>%
    select(-dist.from.trap.mi)
dim(hosp2trap_dist)

#head(hosp2trap_dist)

#hosp2trap_dist_nodup <- hosp2trap_dist[!duplicated(hosp2trap_dist),]
#dim(hosp2trap_dist_nodup)
#dim(hosp2trap_dist)

head(hosp2trap_dist,2)

Joining, by = "dummy"
Warning message:
“package ‘bindrcpp’ was built under R version 3.4.4”

[1] 9118    8

trap_trap_name,loc_lat,loc_lng,cfname,longitude,latitude,cftype,dist_round_mi
T002,41.9563,-87.79752,Cermak Hospital,-87.69757,41.84098,Hospital,9.5
T002,41.9563,-87.79752,Hartgrove Hospital,-87.76369,41.86881,Hospital,6.3


In [8]:
getwd()
setwd("/Users/tedinciong/kaggle/data/00-DATA/hospitals/")
getwd()

#save(hosp2trap_dist, file="hosp2trap_dist.RData", compress = FALSE)
#write.csv(hosp2trap_dist, 'hosp2trap_dist.csv',row.names = FALSE)

[1] "/Users/tedinciong/kaggle/src/jupyter"

[1] "/Users/tedinciong/kaggle/data/00-DATA/hospitals"

### Write out the hospital geospatial info

In [11]:
nodes_hosp_df <- hosp2trap_dist %>%
    select(-c("loc_lat", "loc_lng")) %>%
    rename("id" = "cfname", "dist_to_trap" = "dist_round_mi")
#reorder var names
nodes_hosp_df <- nodes_hosp_df[,c(2:6,1)]
#total obs
dim(nodes_hosp_df)

head(nodes_hosp_df)

[1] 9118    6

id,longitude,latitude,cftype,dist_to_trap,trap_trap_name
Cermak Hospital,-87.69757,41.84098,Hospital,9.5,T002
Hartgrove Hospital,-87.76369,41.86881,Hospital,6.3,T002
Sacred Heart Hospital,-87.70889,41.89184,Hospital,6.4,T002
Saint Mary Of Nazareth Hospital,-87.68454,41.90124,Hospital,7.0,T002
Saint Elizabeth's Hospital,-87.68590,41.90669,Hospital,6.7,T002
Advocate Illinois Masonic Medical Center,-87.65163,41.93813,Hospital,7.6,T002


In [12]:
#check for dups
check_for_dups <- nodes_hosp_df[duplicated(nodes_hosp_df),] # keep unique traps
dim(check_for_dups)


[1] 0 6

In [13]:
#check after dedupped
dedupped <- nodes_hosp_df[!duplicated(nodes_hosp_df),] # keep unique traps
dim(dedupped)

[1] 9118    6

In [15]:
# dedup df
nodes_hosp_df <- nodes_hosp_df[!duplicated(nodes_hosp_df),] # keep unique traps
dim(nodes_hosp_df)


[1] 9118    6

In [16]:
save(nodes_hosp_df, file="nodes_hosp_df.RData", compress = FALSE)
write.csv(nodes_hosp_df, 'nodes_hosp_df.csv',row.names = FALSE)

In [ ]:
#### Write out the trap geospatial info

In [17]:
nodes_trap_df <- hosp2trap_dist %>%
    select(c("trap_trap_name", "loc_lat", "loc_lng")) %>%
    rename("id" = "trap_trap_name","longitude" = "loc_lng", "latitude" = "loc_lat" )
dim(nodes_trap_df)
head(nodes_trap_df)

[1] 9118    3

id,latitude,longitude
T002,41.9563,-87.79752
T002,41.9563,-87.79752
T002,41.9563,-87.79752
T002,41.9563,-87.79752
T002,41.9563,-87.79752
T002,41.9563,-87.79752


In [18]:
#check for dups
check_for_dups <- nodes_trap_df[duplicated(nodes_trap_df),] # keep unique traps
dim(check_for_dups)


[1] 8924    3

In [19]:
#check after dedupped
dedupped <- nodes_trap_df[!duplicated(nodes_trap_df),] # keep unique traps
dim(dedupped)

[1] 194   3

In [20]:
nodes_trap_df <- nodes_trap_df[!duplicated(nodes_trap_df),] # keep unique traps
dim(nodes_trap_df)
save(nodes_trap_df, file="nodes_trap_df.RData", compress = FALSE)
write.csv(nodes_trap_df, 'nodes_trap_df.csv',row.names = FALSE)

[1] 194   3

#### Write out the hospital to trap edges

In [23]:
edges_hosp_df <- hosp2trap_dist[,c(1,4)]

colnames(edges_hosp_df) <- c("from", "to")
head(edges_hosp_df,2)
dim(edges_hosp_df)


from,to
T002,Cermak Hospital
T002,Hartgrove Hospital


[1] 9118    2

In [24]:
check_for_dups <- edges_hosp_df[duplicated(edges_hosp_df),] # keep unique traps
dim(edges_hosp_df)
head(check_for_dups,3)

edges_hosp_df %>%
    filter(from %in% "T007") %>%
    filter(to %in% "Englewood Health Center")

[1] 9118    2

,from,to
26,T002,Englewood Health Center
73,T007,Englewood Health Center
120,T015,Englewood Health Center


from,to
T007,Englewood Health Center
T007,Englewood Health Center


In [25]:
#check after dedupped
dedupped <- edges_hosp_df[!duplicated(edges_hosp_df),] # keep unique traps
dim(dedupped)

[1] 8924    2

In [26]:
edges_hosp_df <- edges_hosp_df[!duplicated(edges_hosp_df),] # keep unique traps
dim(edges_hosp_df)


#h2t_nodes_df <- data.frame("id" = unique(unlist(h2t_edges_df)))

save(edges_hosp_df, file="edges_hosp_df.RData", compress = FALSE)
write.csv(edges_hosp_df, 'edges_hosp_df.csv',row.names = FALSE)

[1] 8924    2